# Index Creation

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Create an Azure AI Search index and load data](#1)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [4]:
import os
from dotenv import load_dotenv

from azure.core.exceptions import ResourceExistsError
from azure.identity import DefaultAzureCredential
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.indexes.models import (
    AzureOpenAIEmbeddingSkill,
    AzureOpenAIParameters,
    AzureOpenAIVectorizer,
    FieldMapping,
    HnswAlgorithmConfiguration,
    HnswParameters,
    IndexProjectionMode,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection,
    SearchIndexerDataSourceType,
    SearchIndexerIndexProjections,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjectionsParameters,
    SearchIndexerSkillset,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    SimpleField,
    SplitSkill,
    VectorSearch,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)
from azure.storage.blob import BlobServiceClient

load_dotenv(override=True)
azure_search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
azure_storage_connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
azure_storage_container = os.getenv("AZURE_STORAGE_CONTAINER_NAME")
azure_openai_embedding_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME")
azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME")
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_embeddings_dimensions = 3072
azure_storage_endpoint = (
    f"https://{os.getenv('AZURE_STORAGE_NAME')}.blob.core.windows.net"
)

<a id='2'></a>
### 1️⃣ Load data to Azure Search

In [ ]:
class Logger:
    def info(self, msg, *args):
        if args:
            print(msg % args)
        else:
            print(msg)


logger = Logger()


def setup_index(
    azure_credential,
    index_name,
    azure_search_endpoint,
    azure_storage_connection_string,
    azure_storage_container,
    azure_openai_embedding_endpoint,
    azure_openai_embedding_deployment,
    azure_openai_embedding_model,
    azure_openai_embeddings_dimensions,
):
    index_client = SearchIndexClient(azure_search_endpoint, azure_credential)
    indexer_client = SearchIndexerClient(azure_search_endpoint, azure_credential)

    data_source_connections = indexer_client.get_data_source_connections()
    if index_name in [ds.name for ds in data_source_connections]:
        logger.info(
            f"Data source connection {index_name} already exists, not re-creating"
        )
    else:
        logger.info(f"Creating data source connection: {index_name}")
        indexer_client.create_data_source_connection(
            data_source_connection=SearchIndexerDataSourceConnection(
                name=index_name,
                type=SearchIndexerDataSourceType.AZURE_BLOB,
                connection_string=azure_storage_connection_string,
                container=SearchIndexerDataContainer(name=azure_storage_container),
            )
        )

    index_names = [index.name for index in index_client.list_indexes()]
    if index_name in index_names:
        logger.info(f"Index {index_name} already exists, not re-creating")
    else:
        logger.info(f"Creating index: {index_name}")
        index_client.create_index(
            SearchIndex(
                name=index_name,
                fields=[
                    SearchableField(
                        name="chunk_id",
                        key=True,
                        analyzer_name="keyword",
                        sortable=True,
                    ),
                    SimpleField(
                        name="parent_id",
                        type=SearchFieldDataType.String,
                        filterable=True,
                    ),
                    SearchableField(name="title"),
                    SearchableField(name="chunk"),
                    SearchField(
                        name="text_vector",
                        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                        vector_search_dimensions=azure_openai_embeddings_dimensions,
                        vector_search_profile_name="vp",
                        stored=True,
                        hidden=False,
                    ),
                ],
                vector_search=VectorSearch(
                    algorithms=[
                        HnswAlgorithmConfiguration(
                            name="algo",
                            parameters=HnswParameters(
                                metric=VectorSearchAlgorithmMetric.COSINE
                            ),
                        )
                    ],
                    vectorizers=[
                        AzureOpenAIVectorizer(
                            name="openai_vectorizer",
                            azure_open_ai_parameters=AzureOpenAIParameters(
                                api_key=azure_openai_api_key,
                                resource_uri=azure_openai_embedding_endpoint,
                                deployment_id=azure_openai_embedding_deployment,
                                model_name=azure_openai_embedding_model,
                            ),
                        )
                    ],
                    profiles=[
                        VectorSearchProfile(
                            name="vp",
                            algorithm_configuration_name="algo",
                            vectorizer="openai_vectorizer",
                        )
                    ],
                ),
                semantic_search=SemanticSearch(
                    configurations=[
                        SemanticConfiguration(
                            name="default",
                            prioritized_fields=SemanticPrioritizedFields(
                                title_field=SemanticField(field_name="title"),
                                content_fields=[SemanticField(field_name="chunk")],
                            ),
                        )
                    ],
                    default_configuration_name="default",
                ),
            )
        )
    skillsets = indexer_client.get_skillsets()
    if index_name in [skillset.name for skillset in skillsets]:
        logger.info(f"Skillset {index_name} already exists, not re-creating")
    else:
        logger.info(f"Creating skillset: {index_name}")
        indexer_client.create_skillset(
            skillset=SearchIndexerSkillset(
                name=index_name,
                skills=[
                    SplitSkill(
                        text_split_mode="pages",
                        context="/document",
                        maximum_page_length=2000,
                        page_overlap_length=500,
                        inputs=[
                            InputFieldMappingEntry(
                                name="text", source="/document/content"
                            )
                        ],
                        outputs=[
                            OutputFieldMappingEntry(
                                name="textItems", target_name="pages"
                            )
                        ],
                    ),
                    AzureOpenAIEmbeddingSkill(
                        context="/document/pages/*",
                        resource_uri=azure_openai_embedding_endpoint,
                        api_key=azure_openai_api_key,
                        deployment_id=azure_openai_embedding_deployment,
                        model_name=azure_openai_embedding_model,
                        dimensions=azure_openai_embeddings_dimensions,
                        inputs=[
                            InputFieldMappingEntry(
                                name="text", source="/document/pages/*"
                            )
                        ],
                        outputs=[
                            OutputFieldMappingEntry(
                                name="embedding", target_name="text_vector"
                            )
                        ],
                    ),
                ],
                index_projections=SearchIndexerIndexProjections(
                    selectors=[
                        SearchIndexerIndexProjectionSelector(
                            target_index_name=index_name,
                            parent_key_field_name="parent_id",
                            source_context="/document/pages/*",
                            mappings=[
                                InputFieldMappingEntry(
                                    name="chunk", source="/document/pages/*"
                                ),
                                InputFieldMappingEntry(
                                    name="text_vector",
                                    source="/document/pages/*/text_vector",
                                ),
                                InputFieldMappingEntry(
                                    name="title",
                                    source="/document/metadata_storage_name",
                                ),
                            ],
                        )
                    ],
                    parameters=SearchIndexerIndexProjectionsParameters(
                        projection_mode=IndexProjectionMode.SKIP_INDEXING_PARENT_DOCUMENTS
                    ),
                ),
            )
        )

    indexers = indexer_client.get_indexers()
    if index_name in [indexer.name for indexer in indexers]:
        logger.info(f"Indexer {index_name} already exists, not re-creating")
    else:
        indexer_client.create_indexer(
            indexer=SearchIndexer(
                name=index_name,
                data_source_name=index_name,
                skillset_name=index_name,
                target_index_name=index_name,
                field_mappings=[
                    FieldMapping(
                        source_field_name="metadata_storage_name",
                        target_field_name="title",
                    )
                ],
            )
        )


def upload_documents(
    document_name,
    azure_credential,
    index_name,
    azure_search_endpoint,
    azure_storage_endpoint,
    azure_storage_container,
):
    indexer_client = SearchIndexerClient(azure_search_endpoint, azure_credential)
    # Upload the documents in /data folder to the blob storage container
    blob_client = BlobServiceClient.from_connection_string(
        azure_storage_connection_string, credential=azure_credential
    )
    container_client = blob_client.get_container_client(azure_storage_container)
    try:
        container_client.create_container()
    except ResourceExistsError:
        pass
    existing_blobs = [blob.name for blob in container_client.list_blobs()]

    for file in os.scandir("../data"):
        if file.name == document_name:
            with open(file.path, "rb") as opened_file:
                filename = os.path.basename(file.path)
                # Check if blob already exists
                if filename in existing_blobs:
                    logger.info("Blob already exists, skipping file: %s", filename)
                else:
                    logger.info("Uploading blob for file: %s", filename)
                    blob_client = container_client.upload_blob(
                        filename, opened_file, overwrite=True
                    )

    # Start the indexer
    try:
        indexer_client.run_indexer(index_name)
        logger.info(
            "Indexer started. Any unindexed blobs should be indexed in a few minutes, check the Azure Portal for status."
        )
    except ResourceExistsError:
        logger.info("Indexer already running, not starting again")

In [ ]:
azure_credential = DefaultAzureCredential()

index_dict = {
    "score": "Score.pdf",
    "dividas": "Dividas.md",
    "cartaowill": "CartaoWill.md",
    "cartaosantander": "CartaoSantander.md",
    "cartaopicpay": "CartaoPicPay.md",
    "cartaoneon": "CartaoNeon.md",
    "cartaoitau": "CartaoItau.md",
    "cartaodm": "CartaoDm.md",
    "cartaodigio": "CartaoDigio.md",
}

# Loop through the dictionary and create an index for each entry
for index_name, document_name in index_dict.items():
    logger.info(f"Setting up index for {index_name}")
    setup_index(
        azure_credential,
        index_name,
        azure_search_endpoint,
        azure_storage_connection_string,
        azure_storage_container,
        azure_openai_embedding_endpoint,
        azure_openai_embedding_deployment,
        azure_openai_embedding_model,
        azure_openai_embeddings_dimensions,
    )

    upload_documents(
        document_name,
        azure_credential,
        index_name,
        azure_search_endpoint,
        azure_storage_endpoint,
        azure_storage_container,
    )

    logger.info(f"Index for {index_name} setup complete")

Setting up index for score
Data source connection score already exists, not re-creating
Index score already exists, not re-creating
Skillset score already exists, not re-creating
Indexer score already exists, not re-creating
